In [26]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import os

In [27]:
# Load the data
df = pd.read_csv('data/gencode.v28.annotation.gtf', sep='\t', comment='#', header=None)
df.columns = ['chr', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']

# Extract key-value pairs from the 'attributes' column
attributes_df = df['attributes'].str.extract(r'gene_id "(.*?)";.*?gene_type "(.*?)";.*?gene_name "(.*?)";.*?level (\d+);')
attributes_df.columns = ['gene_id', 'gene_type', 'gene_name', 'level']

# Concatenate the original DataFrame with the new attributes DataFrame
df = pd.concat([df.iloc[:, :-1], attributes_df], axis=1)

df.head()


,chr,source,type,start,end,score,strand,phase,gene_id,gene_type,gene_name,level
0,chr1,HAVANA,gene,11869,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2
1,chr1,HAVANA,transcript,11869,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2
2,chr1,HAVANA,exon,11869,12227,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2
3,chr1,HAVANA,exon,12613,12721,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2
4,chr1,HAVANA,exon,13221,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2


In [28]:
transcripts = df[df['type'] == 'transcript']
transcripts['start'] = transcripts['start'] - 1500
transcripts['end'] = transcripts['start'] + 1500

# ensure start positions are not less than 1
transcripts['start'] = transcripts['start'].apply(lambda x: max(1, x))

/var/folders/0m/90sjqm9d78l50t2x2lmdrztm0000gn/T/ipykernel_28069/929654069.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcripts['start'] = transcripts['start'] - 1500
/var/folders/0m/90sjqm9d78l50t2x2lmdrztm0000gn/T/ipykernel_28069/929654069.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcripts['end'] = transcripts['start'] + 1500
/var/folders/0m/90sjqm9d78l50t2x2lmdrztm0000gn/T/ipykernel_28069/929654069.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [34]:
# Randomly select half of the promoters
selected_promoters = transcripts.sample(frac=0.5)

# Set reads to 0 for the other half
transcripts.loc[~transcripts.index.isin(selected_promoters.index), 'reads'] = 0

# Load the genome sequence
genome = SeqIO.to_dict(SeqIO.parse('data/genome.fa', 'fasta'))

counter = 0

os.makedirs("output/no_reads", exist_ok=True)
os.makedirs("output/with_reads", exist_ok=True)

for index, row in transcripts.iterrows():
    # Generate artificial 'reads' peak
    if index in selected_promoters.index:
        reads = np.random.poisson(10, (row['end'] - row['start'] + 1))
        descriptor = 'with_reads'
    else:
        reads = np.zeros((row['end'] - row['start'] + 1))
        descriptor = 'no_reads'
    
    # Extract genome sequence for the promoter region
    chrom = row['chr']
    start = row['start']
    end = row['end']
    sequence = genome[chrom][start-1:end]
    
    # Create a DataFrame for the promoter region
    promoter_df = pd.DataFrame({
        'chrom': chrom,
        'pos': np.arange(start, end+1),
        'base': list(sequence),
        'reads': reads,
    })
    
    # Save the promoter DataFrame to a file
    file_name = f'output/{descriptor}/promoter_{index}.feather'
    promoter_df.to_feather(file_name)
